# 🏆 Capstone Project: InfoSynthesizer
## The Context-Aware Research Assistant

**Track:** Concierge Agents
**Author:** Saravana Prashanth K

---

### Project Overview
**The Problem:** Professionals waste hours switching between technical documentation and market news to understand a new topic. Furthermore, generic AI summaries don't know *who* you are—giving the same explanation to a CEO as they do to a Developer.

**The Solution:** **InfoSynthesizer**. A multi-agent system that:
1.  **Reads your Persona:** Uses Memory to understand your job title and interests.
2.  **Parallellizes Research:** Uses a `ParallelAgent` to scout Technical and Market news simultaneously.
3.  **Synthesizes:** Uses a Writer agent to compile a tailored executive brief.

### Key Concepts Applied
This project demonstrates mastery of the following **5-Day Intensive** concepts:
1.  **Multi-Agent Architectures:** Using `SequentialAgent` orchestrating a nested `ParallelAgent`.
2.  **Tools:** Utilizing `Google Search` for real-time grounding.
3.  **State & Memory:** Implementing `InMemorySessionService` and `preload_memory` for context-aware responses.

In [1]:
# --- STEP 1: SETUP & AUTHENTICATION ---
# 1. Authenticate using Kaggle Secrets

import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Authentication Successful")
except Exception as e:
    print(f"❌ Error: {e}. Please ensure 'GOOGLE_API_KEY' is in your Add-ons -> Secrets.")

✅ Authentication Successful


In [2]:
# Importing ADK components

import os
import asyncio
from kaggle_secrets import UserSecretsClient
from google.genai import types
from google.adk.agents import Agent, LlmAgent, ParallelAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import google_search, preload_memory

In [3]:
# 2. Configure the Model

retry_config = types.HttpRetryOptions(attempts=3, exp_base=2, initial_delay=1, http_status_codes=[429, 500, 503])
model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)
print("✅ Model Configured")

✅ Model Configured


### Agent Architecture

To solve this problem efficiently, we aren't just using one agent. We are building a team.

**The Team Structure:**
1.  **ContextManager (LlmAgent):** The "Memory Handler". It looks at past interactions to define *who* the user is.
2.  **ResearchTeam (ParallelAgent):**
    * *TechResearcher:* Googles technical specs and docs.
    * *MarketResearcher:* Googles stock trends and business news.
    * *Why Parallel?* These tasks don't depend on each other. Running them together cuts latency in half.
3.  **Writer (Agent):** Takes the context + tech data + market data and writes the final report.

In [4]:
# --- STEP 2: DEFINE SPECIALIST AGENTS ---

# 1. User Context Agent (The "Memory" Handler)
context_agent = LlmAgent(
    name="ContextManager",
    model=model,
    instruction="You are a context manager. Use 'preload_memory' to find out details about the user (e.g., their job, interests). Summarize who the user is in one sentence so the researchers know how to tailor the content.",
    tools=[preload_memory]
)

In [5]:
# 2. Parallel Research Team (The "Workers")

tech_researcher = Agent(
    name="TechResearcher",
    model=model,
    instruction="Search for the latest TECHNICAL developments regarding the user's query. Focus on specs, code, architecture, and release notes.",
    tools=[google_search],
    output_key="tech_findings" # Saving state for the writer later
)

market_researcher = Agent(
    name="MarketResearcher",
    model=model,
    instruction="Search for the latest MARKET/BUSINESS news regarding the user's query. Focus on stock prices, acquisitions, competitor analysis, and business impact.",
    tools=[google_search],
    output_key="market_findings"
)

In [6]:
# Group them into a Parallel Agent to run simultaneously
research_team = ParallelAgent(
    name="ResearchTeam",
    sub_agents=[tech_researcher, market_researcher]
)

In [7]:
# 3. Writer Agent (The "Synthesizer")
writer_agent = Agent(
    name="Writer",
    model=model,
    instruction="""
    You are an executive writer. 
    1. Read the User Context provided by the ContextManager.
    2. Read the Tech Findings: {tech_findings}
    3. Read the Market Findings: {market_findings}
    
    Synthesize a cohesive report answering the user's original query. 
    
    IMPORTANT: Tailor the tone specifically to the user's background found in the context. 
    If they are technical, use jargon. If they are business-focused, focus on ROI.
    
    Format the output in clean Markdown with bold headers.
    """,
)

print("✅ Agents Initialised")

✅ Agents Initialised


In [8]:
# --- STEP 3: ORCHESTRATION ---

root_agent = SequentialAgent(
    name="InfoSynthesizer",
    sub_agents=[context_agent, research_team, writer_agent]
)

In [9]:
# --- STEP 4: RUNTIME & MEMORY ---

session_service = InMemorySessionService()
memory_service = InMemoryMemoryService() 

runner = Runner(
    agent=root_agent,
    app_name="InfoSynthesizerApp",
    session_service=session_service,
    memory_service=memory_service
)

print("✅ Workflow Orchestrated & Runner Ready")

✅ Workflow Orchestrated & Runner Ready


### 🚀 Live Demonstration

In this demo, we will simulate a realistic usage pattern:
1.  **Memory Seeding:** We will tell the system *who* we are (simulating a previous conversation).
2.  **The Task:** We will ask a complex question about **"Gemini 1.5"**.
3.  **The Result:** Watch how the agent finds both technical specs and business news, then writes a report tailored to a **Data Scientist**.

In [10]:
async def run_demo():
    print("--- 🧠 PHASE 1: MEMORY INGESTION ---")
    user_id = "user1"
    intro_session_id = "intro_session"
    task_session_id = "task_session" # Define the new session ID
    
    # 1. Create the intro session
    await session_service.create_session(
        app_name="InfoSynthesizerApp", 
        user_id=user_id, 
        session_id=intro_session_id
    )
    
    # We simulate the user introducing themselves.
    persona_text = "I am a Marketing Executive interested in brand safety and ROI."
    print(f"User: '{persona_text}'")
    
    # Run the agent silently to populate the session history
    async for event in runner.run_async(
        user_id=user_id, 
        session_id=intro_session_id, 
        new_message=types.Content(parts=[types.Part(text=persona_text)])
    ):
        pass 

    print("System: Storing persona in memory...")
    
    # 2. Retrieve the populated session object
    session = await session_service.get_session(
        app_name="InfoSynthesizerApp", 
        user_id=user_id, 
        session_id=intro_session_id
    )
    
    # 3. Add the session history to long-term memory
    await memory_service.add_session_to_memory(session)
    
    print("\n--- 🚀 PHASE 2: EXECUTING THE TASK ---")
    query = "Tell me about the latest updates in Gemini 1.5."
    
    print(f"User Query: {query}")
    print("...Agent is thinking (running Context check, Parallel Research, and Drafting)...")
    print("...This might take 10-20 seconds as it browses the web...")
    
    await session_service.create_session(
        app_name="InfoSynthesizerApp", 
        user_id=user_id, 
        session_id=task_session_id
    )
    
    # 4. Run the actual task in the NEW session
    async for event in runner.run_async(
        user_id=user_id, 
        session_id=task_session_id, 
        new_message=types.Content(parts=[types.Part(text=query)])
    ):
        if event.is_final_response() and event.content:
             print(f"\n🤖 FINAL REPORT:\n\n{event.content.parts[0].text}")

# Execute the demo
await run_demo()

--- 🧠 PHASE 1: MEMORY INGESTION ---
User: 'I am a Marketing Executive interested in brand safety and ROI.'
System: Storing persona in memory...

--- 🚀 PHASE 2: EXECUTING THE TASK ---
User Query: Tell me about the latest updates in Gemini 1.5.
...Agent is thinking (running Context check, Parallel Research, and Drafting)...
...This might take 10-20 seconds as it browses the web...

🤖 FINAL REPORT:

The user is a Marketing Executive interested in brand safety and ROI, and has previously received information about AI's role in these areas.

🤖 FINAL REPORT:

Gemini 1.5 Pro continues to be a rapidly evolving AI model, with recent updates focusing on enhanced capabilities, improved performance, and broader accessibility for businesses. Key developments include significant price reductions for API usage, making advanced AI more cost-effective for marketing applications.

Here are some of the latest updates and their impact for a Marketing Executive focused on brand safety and ROI:

**Enhanced 